In [1]:
!pwd

/home/dell/af/Pred_props


In [2]:
from unimol_tools import MolTrain, MolPredict
import pandas as pd
import numpy as np
import math
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import spearmanr
import os

2024-09-20 16:17:02 | unimol_tools/weights/weighthub.py | 17 | INFO | Uni-Mol Tools | Weights will be downloaded to default directory: /home/dell/miniconda3/envs/unimol/lib/python3.10/site-packages/unimol_tools-0.1.0.post2-py3.10.egg/unimol_tools/weights


In [3]:
# 更改工作目录到 `af` 目录
os.chdir('/home/dell/af')

# 验证当前工作目录
print(os.getcwd())

/home/dell/af


In [4]:
weights_path = r"./Unimol_weights/HOMO/0822_dataset_lr_1e-4"
data_path = r"./Data4pred/mol_step_screening.csv"
save_path = r"./Data4pred/mol_step_screening_HOMO.csv"
input_csv = r"./tmp.csv"

In [5]:
data = pd.read_csv(data_path)
data.head(5)

,SMILES
0,O=P(O)(O)CCCCn1c2ccccc2c2cccc(O)c21
1,O=P(O)(O)CCn1c2ccccc2c2cccc(O)c21
2,O=P(O)(O)CCCn1c2ccccc2c2cccc(O)c21
3,Cn1c2ccccc2c2c(C=CCP(=O)(O)O)cccc21
4,O=P(O)(O)CCCn1c2ccccc2c2c3ccccc3ccc21


In [6]:
pd.DataFrame(data.iloc[:,0]).to_csv(input_csv, index=False)

In [7]:
clf = MolPredict(load_model=weights_path)

In [8]:
# 基于SMILES的文件输入模式的预测 
test_pred = clf.predict(data=input_csv)

2024-09-20 16:17:14 | unimol_tools/data/conformer.py | 89 | INFO | Uni-Mol Tools | Start generating conformers...
45it [00:01, 25.92it/s]
2024-09-20 16:17:16 | unimol_tools/data/conformer.py | 93 | INFO | Uni-Mol Tools | Failed to generate conformers for 0.00% of molecules.
2024-09-20 16:17:16 | unimol_tools/data/conformer.py | 95 | INFO | Uni-Mol Tools | Failed to generate 3d conformers for 0.00% of molecules.
2024-09-20 16:17:17 | unimol_tools/models/unimol.py | 120 | INFO | Uni-Mol Tools | Loading pretrained weights from /home/dell/miniconda3/envs/unimol/lib/python3.10/site-packages/unimol_tools-0.1.0.post2-py3.10.egg/unimol_tools/weights/mol_pre_no_h_220816.pt
2024-09-20 16:17:19 | unimol_tools/models/nnmodel.py | 206 | INFO | Uni-Mol Tools | start predict NNModel:unimolv1
2024-09-20 16:17:21 | unimol_tools/tasks/trainer.py | 300 | INFO | Uni-Mol Tools | load model success!
2024-09-20 16:17:24 | unimol_tools/tasks/trainer.py | 300 | INFO | Uni-Mol Tools | load model success!
2024-0

In [9]:
os.remove(input_csv)

In [10]:
test_results = pd.DataFrame({
    "smiles": clf.datahub.data["smiles"],
    "pred": test_pred.flatten(),
    })
test_results.head(10)

,smiles,pred
0,O=P(O)(O)CCCCn1c2ccccc2c2cccc(O)c21,-5.142279
1,O=P(O)(O)CCn1c2ccccc2c2cccc(O)c21,-5.218646
2,O=P(O)(O)CCCn1c2ccccc2c2cccc(O)c21,-5.175689
3,Cn1c2ccccc2c2c(C=CCP(=O)(O)O)cccc21,-5.371743
4,O=P(O)(O)CCCn1c2ccccc2c2c3ccccc3ccc21,-5.182746
5,O=P(O)(O)CCn1c2ccccc2c2c3ccccc3ccc21,-5.231421
6,O=P(O)(O)CCCCn1c2ccccc2c2cc3ccccc3cc21,-5.112527
7,Cn1c2ccccc2c2cc(C=CCP(=O)(O)O)ccc21,-5.248266
8,Cn1c2ccccc2c2cc(C=NCCP(=O)(O)O)ccc21,-5.464944
9,O=P(O)(O)CCCCOc1ccc2c(c1)c1ccccc1n2-c1ccccc1,-5.088068


In [11]:
test_results.to_csv(save_path, index=False)